In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Load the image
image_path = "tr.jpeg"
image = Image.open(image_path)
data = np.array(image)

# Display the image
plt.imshow(data)
plt.title("Original Image")
plt.axis('off')
plt.show()

In [ ]:
extent = data.shape
n_streaks = 50000
streaks = np.random.rand(n_streaks, 5)
streaks[:,0] = (streaks[:,0]*extent[0]).astype(int) # x
streaks[:,1] = (streaks[:,1]*extent[1]).astype(int) # y
streaks[:,2] = streaks[:,2]*2*np.pi  # direction in radians
streaks[:,3] = (streaks[:,3]*10).astype(int)+2 # length in px
streaks[:,4] = streaks[:,4]/np.log(streaks[:,3]+1) # reduce streak brightness
for streak in streaks:
    x, y = streak[0], streak[1]
    data[int(np.floor(x)), int(np.floor(y))] = streak[4]
    for l in range(int(streak[3])):
        x += np.cos(streak[2])
        y += np.sin(streak[2])
        try:
            data[int(np.floor(x)), int(np.floor(y))] = streak[4] 
        except IndexError:
            pass
streaked_image = Image.fromarray(data)
streaked_image.save(output_path)
plt.imshow(data, cmap='gray')
plt.title("Streaked Image")
plt.axis('off')
plt.show()

In [ ]:
from scipy.signal import convolve2d
from scipy.stats import norm

# Ensure the image is in grayscale (2D array)
if len(data.shape) == 3:  # if it's a 3D array, convert to 2D by selecting one channel
    data = data.mean(axis=2)

# Create the kernel
y, x = np.mgrid[-4:5, -4:5]
kernel = norm.pdf(x) * norm.pdf(y)

# Convolve the image with the kernel
data_convolved = convolve2d(data, kernel, boundary='wrap', mode='same')

# Display the convolved image
plt.imshow(data_convolved, cmap='gray')
plt.title("Convolved Image")
plt.axis('off')
plt.show()


In [ ]:
# Detect Bright Spots (Sources)
from scipy.ndimage import label, find_objects

# Set a threshold to identify bright sources (assuming stars are bright spots)
threshold = data_convolved.mean() + data_convolved.std()  # this can be adjusted

# Create a binary map of sources
binary_sources = data_convolved > threshold

# Label the sources in the binary map
labeled_sources, num_features = label(binary_sources)

# Find the slices that define the bounding box of each source
slices = find_objects(labeled_sources)

# Extract the center coordinates of each detected source
sources = [( (slc[0].start + slc[0].stop) // 2, (slc[1].start + slc[1].stop) // 2 ) for slc in slices]

source_thumbs = []
for source in sources:
    try:
        x, y = source[0], source[1]
        thumb_data = data[int(x)-7:int(x)+8, 
                          int(y)-7:int(y)+8]
        if thumb_data.shape == (15,15):
            source_thumbs.append(thumb_data)
    except ValueError:
        pass
    
source_thumbs = np.array(source_thumbs)
print(len(source_thumbs), 'stars extracted')


In [ ]:
from astropy.visualization import ImageNormalize, MinMaxInterval, LinearStretch
thumb = source_thumbs[3]

norm = ImageNormalize(thumb, interval=MinMaxInterval(), stretch=LinearStretch())

f, ax = plt.subplots(figsize=(10,10))
im = ax.imshow(thumb, norm=norm, origin='lower', cmap='gray')
f.colorbar(im)
plt.show()


In [ ]:
streak_thumbs = []
for streak in streaks:
    x, y = streak[0], streak[1]
    x += streak[3]/2 * np.cos(streak[2])
    y += streak[3]/2 * np.sin(streak[2])
    try:
        thumb_data = data[int(x)-7:int(x)+8, 
                          int(y)-7:int(y)+8]
        if thumb_data.shape == (15,15):
            streak_thumbs.append(thumb_data)
    except ValueError:
        pass
    
#streak_thumbs    
streak_thumbs = np.array(streak_thumbs)
print(len(streak_thumbs), 'streaks extracted')

In [ ]:
import numpy as np
import cv2
from sklearn.ensemble import RandomForestClassifier
from scipy.ndimage import label, find_objects

# Step 1: Object Detection
threshold = data_gray.mean() + data_gray.std()
binary_objects = data_gray > threshold
labeled_objects, num_objects = label(binary_objects)

# Step 2: Feature Extraction
features = []
labels = []

for slc in find_objects(labeled_objects):
    y, x = (slc[0].start + slc[0].stop) // 2, (slc[1].start + slc[1].stop) // 2
    obj = data_gray[slc]

    # Feature 1: Size (area)
    area = np.sum(binary_objects[slc])

    # Feature 2: Mean Brightness
    mean_brightness = np.mean(obj)

    # Feature 3: Aspect Ratio (Shape)
    aspect_ratio = obj.shape[1] / obj.shape[0]

    # Feature 4: Eccentricity (to distinguish stars from galaxies)
    contours, _ = cv2.findContours(binary_objects[slc].astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Initialize eccentricity
    eccentricity = 0
    
    if contours:
        contour = contours[0]
        if len(contour) >= 5:  # Ensure there are enough points to fit an ellipse
            ellipse = cv2.fitEllipse(contour)
            eccentricity = np.sqrt(1 - (ellipse[1][1] / ellipse[1][0])**2) if ellipse[1][0] != 0 else 0
        else:
            # Use bounding box aspect ratio as a fallback
            x, y, w, h = cv2.boundingRect(contour)
            eccentricity = h / w if w != 0 else 0

    # Construct feature vector
    feature_vector = [area, mean_brightness, aspect_ratio, eccentricity]
    features.append(feature_vector)

    # Classification (manual for now; can automate with labeled data)
    if aspect_ratio > 1.5:
        labels.append('Streak')  # Streak
    elif area > 100 and eccentricity > 0.5:
        labels.append('Galaxy')  # Galaxy
    else:
        labels.append('Star')  # Star

features = np.array(features)
labels = np.array(labels)

# Step 3: Classification (If automated with labeled data)
# For demonstration, labels are manually assigned in the above step.

# Step 4: Taxonomy Creation
# Create a simple taxonomy structure

taxonomy = {
    'Astronomical Objects': {
        'Stars': np.sum(labels == 'Star'),
        'Galaxies': np.sum(labels == 'Galaxy'),
        'Streaks': np.sum(labels == 'Streak'),
    }
}

# Step 5: Visualization and Reporting
print("Taxonomy of Detected Objects:")
for category, count in taxonomy['Astronomical Objects'].items():
    print(f"{category}: {count} objects detected")

# Optionally, you can visualize each category separately
for category in ['Star', 'Galaxy', 'Streak']:
    plt.figure(figsize=(10, 10))
    plt.imshow(data_gray, cmap='gray')
    plt.title(f"Detected {category}s")

    for i, lbl in enumerate(labels):
        if lbl == category:
            y, x = (find_objects(labeled_objects)[i][0].start + find_objects(labeled_objects)[i][0].stop) // 2, \
                   (find_objects(labeled_objects)[i][1].start + find_objects(labeled_objects)[i][1].stop) // 2
            plt.plot(x, y, 'ro' if category == 'Star' else 'bo' if category == 'Galaxy' else 'go', markersize=5)

    plt.axis('off')
    plt.show()


In [ ]:
thumb=streak_thumbs[2]

norm = ImageNormalize(thumb, interval=MinMaxInterval(), stretch=LinearStretch())

f, ax = plt.subplots(figsize=(10,10))
im = ax.imshow(thumb, norm=norm, origin='lower',  cmap='gray')
f.colorbar(im)

In [ ]:
from sklearn.model_selection import train_test_split

target = np.append(np.zeros(len(source_thumbs)), np.ones(len(streak_thumbs)))
thumbs = np.append(source_thumbs, streak_thumbs, axis=0)

x_train, x_test, y_train, y_test = train_test_split(thumbs, target, test_size=0.2, random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout
from keras.optimizers import RMSprop
from keras.utils import to_categorical

In [ ]:
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=5, activation='relu', strides=1, input_shape=(15, 15, 1), padding='same'))
model.add(Conv2D(filters=32, kernel_size=3, activation='relu', strides=1))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(20, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])
model.summary()

model.fit(x_train.reshape((-1, 15, 15, 1)), to_categorical(y_train), epochs=5, batch_size=64)

In [ ]:
loss, acc = model.evaluate(x_test.reshape((-1, 15, 15, 1)), to_categorical(y_test), batch_size=32)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))

In [ ]:
# Visualize more of the extracted thumbnails
n_thumbs_to_show = 100  # number of thumbnails to display
thumbs_to_show = source_thumbs[:n_thumbs_to_show]

# Set up the plot for more thumbnails
fig, axes = plt.subplots(4, 5, figsize=(15, 15))  # 4x5 grid

for i, ax in enumerate(axes.flatten()):
    ax.imshow(thumbs_to_show[i], cmap='gray')
    ax.axis('off')
    ax.set_title(f'Thumbnail {i+1}')

plt.tight_layout()
plt.show()


In [ ]:
# Since the model training and test sets were mentioned, but not actually executed in this session, 
# I'll simulate predictions and true labels to demonstrate visualization.
import matplotlib.pyplot as plt
import numpy as np

# Simulating the x_test, predicted_classes, and true_classes for demonstration purposes
n_images = 20  # Number of images to display

# Create dummy data for demonstration
x_test = np.random.rand(n_images, 15, 15)
predicted_classes = np.random.randint(0, 2, n_images)
true_classes = np.random.randint(0, 2, n_images)

# Visualize some of the test images with their predictions and true labels
fig, axes = plt.subplots(1, n_images, figsize=(20, 5))

for i in range(n_images):
    ax = axes[i]
    ax.imshow(x_test[i], cmap='gray')
    ax.axis('off')
    ax.set_title(f"Pred: {predicted_classes[i]}\nTrue: {true_classes[i]}")

plt.show()


In [ ]:
import cv2
import numpy as np
import csv

# Convert image to grayscale (if not already)
if len(data.shape) == 3:  # if it's a 3D array, convert to 2D by selecting one channel
    data_gray = data.mean(axis=2)
else:
    data_gray = data

# Apply Gaussian blur to reduce noise
blurred = cv2.GaussianBlur(data_gray, (5, 5), 0)

# Edge detection using Canny
edges = cv2.Canny(blurred.astype(np.uint8), threshold1=50, threshold2=150, apertureSize=3)

# Apply Hough Line Transform to detect lines
lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=50, minLineLength=50, maxLineGap=10)

# Collect the coordinates of detected streaks
streak_coordinates = []
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        streak_coordinates.append((x1, y1, x2, y2))

# Save the coordinates to a CSV file
output_file = 'streak_coordinates.csv'
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["x1", "y1", "x2", "y2"])  # Header
    writer.writerows(streak_coordinates)

print(f"Detected streak coordinates saved to {output_file}")


In [ ]:
import matplotlib.pyplot as plt
from scipy.ndimage import label, find_objects

# Detect the stars using a thresholding technique
threshold = data_convolved.mean() + data_convolved.std()
binary_sources = data_convolved > threshold
labeled_sources, num_features = label(binary_sources)
slices = find_objects(labeled_sources)

# Extract the center coordinates of each detected source
sources = [( (slc[0].start + slc[0].stop) // 2, (slc[1].start + slc[1].stop) // 2 ) for slc in slices]

# Visualize the detected stars on the original or convolved image
plt.figure(figsize=(10, 10))
plt.imshow(data_convolved, cmap='gray')
plt.title("Detected Stars")

# Overlay the detected stars with red circles
for y, x in sources:
    plt.plot(x, y, 'ro', markersize=5, markeredgewidth=0.5, markerfacecolor='none')

plt.axis('off')
plt.show()


In [ ]:
!pip install opencv-python

In [ ]:
!pip install opencv-python-headless

In [ ]:
import numpy as np
import cv2
from sklearn.ensemble import RandomForestClassifier
from scipy.ndimage import label, find_objects

# Step 1: Object Detection
threshold = data_gray.mean() + data_gray.std()
binary_objects = data_gray > threshold
labeled_objects, num_objects = label(binary_objects)

# Step 2: Feature Extraction
features = []
labels = []

for slc in find_objects(labeled_objects):
    y, x = (slc[0].start + slc[0].stop) // 2, (slc[1].start + slc[1].stop) // 2
    obj = data_gray[slc]

    # Feature 1: Size (area)
    area = np.sum(binary_objects[slc])

    # Feature 2: Mean Brightness
    mean_brightness = np.mean(obj)

    # Feature 3: Aspect Ratio (Shape)
    aspect_ratio = obj.shape[1] / obj.shape[0]

    # Feature 4: Eccentricity (to distinguish stars from galaxies)
    contours, _ = cv2.findContours(binary_objects[slc].astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Initialize eccentricity
    eccentricity = 0
    
    if contours:
        contour = contours[0]
        if len(contour) >= 5:  # Ensure there are enough points to fit an ellipse
            ellipse = cv2.fitEllipse(contour)
            eccentricity = np.sqrt(1 - (ellipse[1][1] / ellipse[1][0])**2) if ellipse[1][0] != 0 else 0
        else:
            # Use bounding box aspect ratio as a fallback
            x, y, w, h = cv2.boundingRect(contour)
            eccentricity = h / w if w != 0 else 0

    # Construct feature vector
    feature_vector = [area, mean_brightness, aspect_ratio, eccentricity]
    features.append(feature_vector)

    # Classification (manual for now; can automate with labeled data)
    if aspect_ratio > 1.5:
        labels.append('Streak')  # Streak
    elif area > 100 and eccentricity > 0.5:
        labels.append('Galaxy')  # Galaxy
    else:
        labels.append('Star')  # Star

features = np.array(features)
labels = np.array(labels)

# Step 3: Classification (If automated with labeled data)
# For demonstration, labels are manually assigned in the above step.

# Step 4: Taxonomy Creation
# Create a simple taxonomy structure

taxonomy = {
    'Astronomical Objects': {
        'Stars': np.sum(labels == 'Star'),
        'Galaxies': np.sum(labels == 'Galaxy'),
        'Streaks': np.sum(labels == 'Streak'),
    }
}

# Step 5: Visualization and Reporting
print("Taxonomy of Detected Objects:")
for category, count in taxonomy['Astronomical Objects'].items():
    print(f"{category}: {count} objects detected")

# Optionally, you can visualize each category separately
for category in ['Star', 'Galaxy', 'Streak']:
    plt.figure(figsize=(10, 10))
    plt.imshow(data_gray, cmap='gray')
    plt.title(f"Detected {category}s")

    for i, lbl in enumerate(labels):
        if lbl == category:
            y, x = (find_objects(labeled_objects)[i][0].start + find_objects(labeled_objects)[i][0].stop) // 2, \
                   (find_objects(labeled_objects)[i][1].start + find_objects(labeled_objects)[i][1].stop) // 2
            plt.plot(x, y, 'ro' if category == 'Star' else 'bo' if category == 'Galaxy' else 'go', markersize=5)

    plt.axis('off')
    plt.show()
